In [1]:
import numpy as np
import pylab as plt
import swyft
from swyft import utils
import torch
%load_ext autoreload
%autoreload 2
swyft.set_verbosity(5)

In [2]:
DEVICE = 'cuda:0'

In [3]:
def model(params, sigma = .1):
    z = np.array(list(params.values()))
    x = np.abs(z-0.5)*2  # 0 ... 1
    return dict(x=x)

In [4]:
def noise(obs, params, sigma = 0.1):
    x = obs['x']
    x = x + np.random.randn(*x.shape) * sigma
    return dict(x=x)

In [30]:
zdim=1
params = ["z"+str(i) for i in range(zdim)]
par0 = {k: 0.25 for k in params}
obs0 = noise(model(par0), par0)
obs1=model(par0)  # Asimov data

In [31]:
prior = swyft.Prior({k: ["uniform", 0., 1.] for k in params})

In [32]:
s = swyft.NestedRatios(model, prior, noise = noise, obs = obs0, device = DEVICE, Ninit = 1000, Nmax=20000)

Creating new cache.


In [33]:
s.run(max_rounds=1, train_args = dict(lr_schedule = [1e-3, 1e-4]))

NRE round: R = 1
Number of training samples is N_R = 1000
Additional simulations are required after growing the cache.
Simulate:  11%|█         | 116/1049 [00:00<00:00, 1151.01it/s]

  adding 1049 new samples to simulator cache.


Simulate: 100%|██████████| 1049/1049 [00:00<00:00, 1150.61it/s]
Starting neural network training.
Entering trainloop
lr: 0.001
validation loss = 1.247
validation loss = 1.005
validation loss = 1.069
lr: 0.0001
validation loss = 1.016
validation loss = 0.977
validation loss = 0.9901
Train losses: [1.3488986286623725, 1.0391969084739685, 0.9219338811677078, 0.8856556970497658]
Valid losses: [1.246727466583252, 1.004695475101471, 1.0159064730008442, 0.9770140449206034]
Finished trainloop.
constr_prior_R : prior_R volume = 0.9845 : 1
Volume converged.


In [34]:
indices = s.cache.sample(s._history[-1]['constr_prior'], 100)
points = swyft.Points(indices, s._cache, s._noise)

In [35]:
def test():
    obs = utils.dict_to_tensor_unsqueeze(points[0]['obs'], device = DEVICE)
    f = s._history[-1]['marginals'].ratio.head(obs).detach().cpu().numpy()
    return f

test()

array([[-0.46053484]], dtype=float32)